 # さまざまな教師あり学習：分類

In [1]:
import unittest
import doctest
import os
path = os.path.dirname(os.path.abspath(__file__))

 ## ロジスティック回帰

 ## ランダムフォレスト

 ## アダブースト

In [2]:
doctest.testmod(verbose=True)
unittest.main(argv=[''], verbosity=2, exit=False)

3 items had no tests:
    __main__
    __main__.__VSCODE_compute_hash
    __main__.__VSCODE_wrap_run_cell
0 tests in 3 items.
0 passed and 0 failed.
Test passed.



----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK
